In [1]:
import pandas as pd
import numpy as np
from jqdatasdk import *
#这里需要你在聚宽网注册它们的的jqdata的账户，然后就可以调用api获取数据了
auth('账户名','密码')

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改
auth success 


In [2]:
import re
#import hanlp
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_colwidth",1000)

In [5]:
text ='朱蕾：女，1971年3月出生，美国国籍，美国天普大学会计学硕士，波士顿大学金融学硕士，哥伦比亚大学会计学博士，美国注册会计师，会计学副教授。曾任普华永道会计师事务所高级分析员、AnalysisGroup经理、美国波士顿大学会计学助理教授、上海交通大学高级金融学院副教授。现任复旦大学泛海国际金融学院副教授（终身教职）及上海益民商业集团股份有限公司独立董事。2019年2月至今，担任公司独立董事。。'

In [ ]:

from py2neo import Graph,Node,Relationship,NodeMatcher,RelationshipMatcher
from LAC import LAC
from tqdm import tqdm
lac = LAC(mode='lac')
graph=Graph('http://localhost:7474/db/data/',name='stock',username='neo4j',password='neo4j')
rmatcher = RelationshipMatcher(graph)
import traceback

i = 0

for index,stock in tqdm(finance.run_query(query(finance.STK_COMPANY_INFO).filter(finance.STK_COMPANY_INFO.id>39345)).iterrows()):
    try:
        i += 1
        print('处理第',i,'条：',stock['short_name'])
        stock_code = stock['code']
        stock_name = stock['short_name']
        stock_full_name = stock['full_name']
        stock_english_name = stock['fullname_en']
        register_location = stock['register_location']
        office_address = stock['office_address']
        register_capital = stock['register_capital']
        main_business = stock['main_business']
        province = stock['province']
        city = stock['city']
        industry_1 = stock['industry_1']
        industry_2 = stock['industry_2']
        cpafirm = stock['cpafirm']
        lawfirm = stock['lawfirm']
        stock_node = Node('COMPANY',code=stock_code,name=stock_full_name,short_name=stock_name,english_name=stock_english_name,\
                         register_location=register_location,office_address=office_address,register_capital=register_capital,\
                         main_business=main_business,province=province,city=city,industry_1=industry_1,cpafirm=cpafirm,\
                         lawfirm=lawfirm)
        graph.create(stock_node)
        stock_node = graph.nodes.match('COMPANY',code=stock_code).first()


        #公司任职信息
        for index,manager in finance.run_query(query(finance.STK_MANAGEMENT_INFO).\
                              filter(finance.STK_MANAGEMENT_INFO.code==stock_code).\
                                               order_by(finance.STK_MANAGEMENT_INFO.pub_date)).iterrows():

            name = manager['name']
            person_id = manager['person_id']
            gender = manager['gender']
            birth_year = manager['birth_year']
            highest_degree = manager['highest_degree']
            title = manager['title']
            title_class = manager['title_class']
            on_job = manager['on_job']
            start_date = manager['start_date']
            leave_date = manager['leave_date']
            person_node = graph.nodes.match('PERSON',name=name,person_id=person_id).first()

            if person_node is None:
                graph.create(Node('PERSON',name=name,person_id=person_id,gender=gender,birth_year=birth_year,
                                 highest_degree=highest_degree))
                person_node = graph.nodes.match('PERSON',name=name,person_id=person_id).first()

            manager_relation = rmatcher.match({person_node,stock_node},r_type='SERVED_IN',name='任职',title=title,title_class=title_class,\
                                            on_job=on_job,start_date=str(start_date),leave_date=str(leave_date)).first()

            if manager_relation is None:

                graph.create(Relationship(person_node,'SERVED_IN',stock_node,name='任职',title=title,title_class=title_class,\
                                                on_job=on_job,start_date=str(start_date),leave_date=str(leave_date)))
                manager_relation = rmatcher.match({person_node,stock_node},r_type='SERVED_IN',name='任职',title=title,title_class=title_class,\
                                            on_job=on_job,start_date=str(start_date),leave_date=str(leave_date)).first()

            #个人简历
            if manager['resume'] is not None:
                lac_result = lac.run(manager['resume'])
                for org,tag in zip(*lac_result):
                    if tag =='ORG':
                        org_node = graph.nodes.match(name=org).first()
                        if org_node is None:
                            graph.create(Node('ORGANIZATION',name=org))
                            org_node = graph.nodes.match('ORGANIZATION',name=org).first()

                        org_relation = rmatcher.match({person_node,org_node},r_type='STAYED_IN',name='曾任职或学习机构').first()
                        if org_relation is None:
                            org_relation = Relationship(person_node,'STAYED_IN',org_node,name='曾任职或学习机构')
                            graph.create(org_relation)

        #十大股东
        for index,share_holder in finance.run_query(query(finance.STK_SHAREHOLDER_TOP10).filter(finance.STK_SHAREHOLDER_TOP10.code==stock_code).\
                                  order_by(finance.STK_SHAREHOLDER_TOP10.pub_date.desc()).limit(10)).iterrows():
            shareholder_name = share_holder['shareholder_name']
            shareholder_rank = share_holder['shareholder_rank']
            shareholder_class = share_holder['shareholder_class']
            share_number = share_holder['share_number']
            share_ratio = share_holder['share_ratio']
            share_nature = share_holder['sharesnature']
            shareholder_node = graph.nodes.match(name=shareholder_name).first()

            if shareholder_node is None:
                graph.create(Node('SHAREHOLDER',name=shareholder_name,shareholder_class=shareholder_class))
                shareholder_node = graph.nodes.match('SHAREHOLDER',name=shareholder_name,shareholder_class=shareholder_class).first()

            shareholder_relation = rmatcher.match({shareholder_node,stock_node},r_type='HOLD',name='持股',\
                                                  share_number=share_number,share_ratio=share_ratio).first()
            if shareholder_relation is None:
                shareholder_relation = Relationship(shareholder_node,'HOLD',stock_node,name='持股',\
                                                  share_number=share_number,share_ratio=share_ratio)
                graph.create(shareholder_relation)

        #十大流通股东
        for index,share_holder in finance.run_query(query(finance.STK_SHAREHOLDER_FLOATING_TOP10).filter(finance.STK_SHAREHOLDER_FLOATING_TOP10.code==stock_code).\
                                  order_by(finance.STK_SHAREHOLDER_FLOATING_TOP10.pub_date.desc()).limit(10)).iterrows():
            shareholder_name = share_holder['shareholder_name']
            shareholder_rank = share_holder['shareholder_rank']
            shareholder_class = share_holder['shareholder_class']
            share_number = share_holder['share_number']
            share_ratio = share_holder['share_ratio']
            share_nature = share_holder['sharesnature']
            shareholder_node = graph.nodes.match(name=shareholder_name).first()

            if shareholder_node is None:
                graph.create(Node('SHAREHOLDER',name=shareholder_name,shareholder_class=shareholder_class))
                shareholder_node = graph.nodes.match('SHAREHOLDER',name=shareholder_name,shareholder_class=shareholder_class).first()

            shareholder_relation = rmatcher.match({shareholder_node,stock_node},r_type='FLOAT_HOLD',name='流通股持股',\
                                                  share_number=share_number,share_ratio=share_ratio).first()
            if shareholder_relation is None:
                shareholder_relation = Relationship(shareholder_node,'FLOAT_HOLD',stock_node,name='流通股持股',\
                                                  share_number=share_number,share_ratio=share_ratio)
                graph.create(shareholder_relation)
    except Exception as e:
        traceback.print_exc()
        continue

In [147]:
finance.run_query(query(finance.STK_COMPANY_INFO).filter(finance.STK_COMPANY_INFO.id>39345)).shape

(339, 45)

In [134]:
stock_node = Node('COMPANY',code='1111',name='英力特')

In [73]:
stock_node

Node('COMPANY', code='1111', name='test')

In [135]:
graph.create(stock_node)

In [141]:
graph.nodes.match(name='社保基金').first()

In [107]:
org_node = graph.nodes.match('ORGANIZATION',name='顺德胜利织衣厂').first()
person_node = graph.nodes.match('PERSON',name='何国英').first()

In [113]:
org_relation = Relationship(person_node,'STAYED_IN',org_node,name='曾任职或学习机构')
graph.create(org_relation)

In [115]:
org_relation

STAYED_IN(Node('PERSON', birth_year='1962', gender='m', highest_degree='硕士及研究生', name='何国英', person_id=201301102), Node('ORGANIZATION', name='顺德胜利织衣厂'), name='曾任职或学习机构')

In [114]:
org_relation =rmatcher.match({person_node,org_node},r_type='STAYED_IN',name='曾任职或学习机构').first()

In [6]:
finance.run_query(query(finance.FUND_PORTFOLIO_STOCK).filter(finance.FUND_PORTFOLIO_STOCK.code=="000002").\
order_by(finance.FUND_PORTFOLIO_STOCK.pub_date.desc()).limit(40))

,id,code,period_start,period_end,pub_date,report_type_id,report_type,rank,symbol,name,shares,market_cap,proportion
0,16128446,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,1,601318,中国平安,3337526.0,2.902980e+08,6.04
1,16128447,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,2,002127,南极电商,20744721.0,2.837878e+08,5.90
2,16128448,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,3,000961,中南建设,28897307.0,2.551632e+08,5.30
3,16128449,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,4,600519,贵州茅台,120575.0,2.409088e+08,5.01
4,16128450,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,5,002271,东方雨虹,5668427.0,2.199350e+08,4.57
5,16128451,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,6,600048,保利地产,13082848.0,2.069707e+08,4.30
6,16128452,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,7,000547,航天发展,7244851.0,1.992334e+08,4.14
7,16128453,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,8,002013,中航机电,15888113.0,1.819189e+08,3.78
8,16128454,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,9,000513,丽珠集团,3443820.0,1.394747e+08,2.90
9,16128455,000001,2020-10-01,2020-12-31,2021-01-21,403004,第四季度,10,000858,五粮液,470942.0,1.374444e+08,2.86


In [7]:
finance.run_query(query(finance.FUND_MAIN_INFO))

,id,main_code,name,advisor,trustee,operate_mode_id,operate_mode,underlying_asset_type_id,underlying_asset_type,start_date,end_date
0,1,000001,华夏成长,华夏基金管理有限公司,中国建设银行股份有限公司,401001,开放式基金,402004,混合型,2001-12-18,None
1,2,000003,中海可转债债券,中海基金管理有限公司,中国农业银行股份有限公司,401001,开放式基金,402003,债券型,2013-03-20,None
2,3,000005,嘉实增强信用,嘉实基金管理有限公司,中国工商银行股份有限公司,401001,开放式基金,402003,债券型,2013-03-08,None
3,4,000007,鹏华国企债,鹏华基金管理有限公司,中国工商银行股份有限公司,401001,开放式基金,402003,债券型,2013-03-08,2018-08-08
4,5,000008,嘉实中证500ETF联接,嘉实基金管理有限公司,中国建设银行股份有限公司,401001,开放式基金,402005,基金型,2013-03-22,None
...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,150053,泰达500A,泰达宏利基金管理有限公司,中国银行股份有限公司,401002,封闭式基金,402001,股票型,2011-12-01,2019-11-06
4996,4997,150055,500A,工银瑞信基金管理有限公司,中国银行股份有限公司,401002,封闭式基金,402001,股票型,2012-01-31,2020-02-17
4997,4998,150057,中小300A,长城基金管理有限公司,中国建设银行股份有限公司,401002,封闭式基金,402001,股票型,2012-01-30,2018-08-13
4998,4999,150059,资源A级,银华基金管理股份有限公司,中国银行股份有限公司,401002,封闭式基金,402001,股票型,2011-12-08,2020-09-30


In [9]:
finance.run_query(query(finance.FUND_MAIN_INFO).filter(finance.FUND_MAIN_INFO.id>12815))

,id,main_code,name,advisor,trustee,operate_mode_id,operate_mode,underlying_asset_type_id,underlying_asset_type,start_date,end_date
0,12818,159977,创业板TH,天弘基金管理有限公司,国泰君安证券股份有限公司,401005,ETF,402001,股票型,2019-09-12,None
1,12820,007249,广发均衡养老三年持有混合(FOF),广发基金管理有限公司,中国建设银行股份有限公司,401004,FOF,402004,混合型,2019-09-24,None
2,12823,007868,汇添富汇鑫货币,汇添富基金管理股份有限公司,上海浦东发展银行股份有限公司,401001,开放式基金,402002,货币型,2019-09-10,None
3,12824,007869,汇添富汇鑫货币B,汇添富基金管理股份有限公司,上海浦东发展银行股份有限公司,401001,开放式基金,402002,货币型,2019-09-10,None
4,12825,007873,华宝科技ETF联接,华宝基金管理有限公司,中国工商银行股份有限公司,401001,开放式基金,402005,基金型,2019-08-30,None
...,...,...,...,...,...,...,...,...,...,...,...
3564,28920,011470,东吴新产业精选股票C,东吴基金管理有限公司,中国建设银行股份有限公司,401001,开放式基金,402001,股票型,2021-02-02,None
3565,28921,011482,中银顺宁回报6个月持有期混合,中银基金管理有限公司,招商银行股份有限公司,401001,开放式基金,402004,混合型,None,None
3566,28922,011483,中银顺宁回报6个月持有期混合C,中银基金管理有限公司,招商银行股份有限公司,401001,开放式基金,402004,混合型,None,None
3567,28923,011484,申万菱信宜选混合,申万菱信基金管理有限公司,招商银行股份有限公司,401001,开放式基金,402004,混合型,None,None


In [2]:
from py2neo import Graph,Node,Relationship,NodeMatcher,RelationshipMatcher
from LAC import LAC
from tqdm import tqdm
lac = LAC(mode='lac')
graph=Graph('http://localhost:7474/db/data/',name='stock',username='neo4j',password='neo4j')
rmatcher = RelationshipMatcher(graph)
import traceback

In [10]:
from py2neo import Graph,Node,Relationship,NodeMatcher,RelationshipMatcher
from LAC import LAC
from tqdm import tqdm
lac = LAC(mode='lac')
graph=Graph('http://localhost:7474/db/data/',name='stock',username='neo4j',password='neo4j')
rmatcher = RelationshipMatcher(graph)
import traceback

i = 0

for index,stock in tqdm(finance.run_query(query(finance.STK_COMPANY_INFO).filter(finance.STK_COMPANY_INFO.id>39346)).iterrows()):
    try:
        i += 1
        print('处理第',i,'条：',stock['short_name'])
        stock_code = stock['code']
        stock_name = stock['short_name']
        stock_full_name = stock['full_name']

        province = stock['province']
        city = stock['city']
        industry_1 = stock['industry_1']
        industry_2 = stock['industry_2']
        cpafirm = stock['cpafirm']
        lawfirm = stock['lawfirm']
        stock_node = graph.nodes.match('COMPANY',name=stock_full_name).first()
        
        cpafirm_node = graph.nodes.match('ORGANIZATION',name=cpafirm).first()
        if cpafirm_node is None:
            graph.create(Node('ORGANIZATION',name=cpafirm))
            cpafirm_node = graph.nodes.match('ORGANIZATION',name=cpafirm).first()
        graph.create(Relationship(stock_node,'CPAFIRM',cpafirm_node,name='会计师事务所'))
        
        lawfirm_node = graph.nodes.match('ORGANIZATION',name=lawfirm).first()
        if lawfirm_node is None:
            graph.create(Node('ORGANIZATION',name=lawfirm))
            lawfirm_node = graph.nodes.match('ORGANIZATION',name=lawfirm).first()
        graph.create(Relationship(stock_node,'LAWFIRM',lawfirm_node,name='律师事务所'))
        
        province_node = graph.nodes.match('PROVINCE',name=province).first()
        if province_node is None:
            graph.create(Node('PROVINCE',name=province))
            province_node = graph.nodes.match('PROVINCE',name=province).first()
        graph.create(Relationship(stock_node,'LOCATE_PROVINCE',province_node,name='所属省份'))
        
        city_node = graph.nodes.match('CITY',name=city).first()
        if city_node is None:
            graph.create(Node('CITY',name=city))
            city_node = graph.nodes.match('CITY',name=city).first()
        graph.create(Relationship(stock_node,'LOCATE_CITY',city_node,name='所属城市'))
        
        industry_node1 = graph.nodes.match('INDUSTRY1',name=industry_1).first()
        if industry_node1 is None:
            graph.create(Node('INDUSTRY1',name=industry_1))
            industry_node1 = graph.nodes.match('INDUSTRY1',name=industry_1).first()
        graph.create(Relationship(stock_node,'INDUSTRY1_OF',industry_node1,name='所属一级行业'))
        
        industry_node2 = graph.nodes.match('INDUSTRY2',name=industry_2).first()
        if industry_node2 is None:
            graph.create(Node('INDUSTRY2',name=industry_2))
            industry_node2 = graph.nodes.match('INDUSTRY2',name=industry_2).first()
        graph.create(Relationship(stock_node,'INDUSTRY2_OF',industry_node2,name='所属二级行业'))
        
    except:
        traceback.print_exc()

1it [00:00,  6.37it/s]

处理第 1 条： 粤海饲料
处理第 2 条： 同兴环保


3it [00:00,  7.15it/s]

处理第 3 条： 中国黄金
处理第 4 条： 威迈斯


5it [00:00,  8.19it/s]

...

处理第 337 条： 环洋股份


337it [01:27,  3.81it/s]

处理第 338 条： 富吉瑞


338it [01:27,  4.09it/s]Traceback (most recent call last):
  File "<ipython-input-10-8afee359daaf>", line 31, in <module>
    graph.create(Relationship(stock_node,'CPAFIRM',cpafirm_node,name='会计师事务所'))
  File "C:\Users\A\anaconda3\lib\site-packages\py2neo\data\__init__.py", line 670, in __init__
    Entity.__init__(self, (n[0], self, n[1]), properties)
  File "C:\Users\A\anaconda3\lib\site-packages\py2neo\data\__init__.py", line 313, in __init__
    Walkable.__init__(self, iterable)
  File "C:\Users\A\anaconda3\lib\site-packages\py2neo\data\__init__.py", line 217, in __init__
    _ = node.labels  # ensure not stale
AttributeError: 'NoneType' object has no attribute 'labels'
339it [01:27,  3.86it/s]

处理第 339 条： 卓然股份


In [5]:
q=query(finance.STK_INCOME_STATEMENT.company_name,
        finance.STK_INCOME_STATEMENT.code,
        finance.STK_INCOME_STATEMENT.pub_date,
        finance.STK_INCOME_STATEMENT.start_date,
        finance.STK_INCOME_STATEMENT.end_date,
        finance.STK_INCOME_STATEMENT.total_operating_revenue,
finance.STK_INCOME_STATEMENT.np_parent_company_owners).filter(finance.STK_INCOME_STATEMENT.code=='600519.XSHG',finance.STK_INCOME_STATEMENT.pub_date>='2015-01-01',finance.STK_INCOME_STATEMENT.report_type==0).limit(20)
df=finance.run_query(q)
print(df)

   company_name         code    pub_date  start_date    end_date  \
0   贵州茅台酒股份有限公司  600519.XSHG  2015-04-21  2014-01-01  2014-12-31   
1   贵州茅台酒股份有限公司  600519.XSHG  2015-04-21  2015-01-01  2015-03-31   
2   贵州茅台酒股份有限公司  600519.XSHG  2015-08-28  2015-01-01  2015-06-30   
3   贵州茅台酒股份有限公司  600519.XSHG  2015-10-23  2015-01-01  2015-09-30   
4   贵州茅台酒股份有限公司  600519.XSHG  2016-03-24  2015-01-01  2015-12-31   
5   贵州茅台酒股份有限公司  600519.XSHG  2016-04-21  2016-01-01  2016-03-31   
6   贵州茅台酒股份有限公司  600519.XSHG  2016-08-27  2016-01-01  2016-06-30   
7   贵州茅台酒股份有限公司  600519.XSHG  2016-10-29  2016-01-01  2016-09-30   
8   贵州茅台酒股份有限公司  600519.XSHG  2017-04-15  2016-01-01  2016-12-31   
9   贵州茅台酒股份有限公司  600519.XSHG  2017-04-25  2017-01-01  2017-03-31   
10  贵州茅台酒股份有限公司  600519.XSHG  2017-07-28  2017-01-01  2017-06-30   
11  贵州茅台酒股份有限公司  600519.XSHG  2017-10-26  2017-01-01  2017-09-30   
12  贵州茅台酒股份有限公司  600519.XSHG  2018-03-28  2017-01-01  2017-12-31   
13  贵州茅台酒股份有限公司  600519.XSHG  2018-04-28  2018-0

In [17]:
finance.run_query(query(finance.STK_INCOME_STATEMENT_PARENT).\
                  filter(finance.STK_INCOME_STATEMENT_PARENT.code=='600519.XSHG'))['report_date']

5      2002-12-31
6      2002-06-30
7      2003-06-30
8      2002-09-30
9      2003-09-30
          ...    
145    2020-03-31
146    2020-06-30
147    2020-06-30
148    2020-09-30
149    2020-09-30
Name: report_date, Length: 145, dtype: object

In [31]:
profit_df=finance.run_query(query(finance.STK_INCOME_STATEMENT)\
                  .filter(finance.STK_INCOME_STATEMENT.code=='600519.XSHG',\
                         finance.STK_INCOME_STATEMENT.report_type==0,\
                         finance.STK_INCOME_STATEMENT.report_date.ilike('_____12-31')))\
[['report_date','total_operating_revenue','total_operating_cost','operating_profit','total_profit','income_tax','net_profit']]

In [210]:
profit_df['year'] = profit_df['report_date'].apply(lambda x:str(x.year))

total_operating_revenue = list(round(profit_df['total_operating_revenue']/100000000,2))
total_operating_cost = list(round(profit_df['total_operating_cost']/100000000,2))
operating_profit = list(round(profit_df['operating_profit']/100000000,2))
total_profit = list(round(profit_df['total_profit']/100000000,2))
income_tax = list(round(profit_df['income_tax']/100000000,2))
net_profit = list(round(profit_df['net_profit']/100000000,2))

In [235]:
from pyecharts import options as opts
from pyecharts.charts import Bar,Timeline
from pyecharts.faker import Faker


profit_chart = (
    Bar()
    .add_xaxis(profit_df['year'].to_list())
    .add_yaxis("营业总收入", total_operating_revenue, gap="0%")
    .add_yaxis("营业总成本", total_operating_cost, gap="0%")
    #.add_yaxis("营业利润", operating_profit, gap="0%")
    .add_yaxis("总利润", total_profit, gap="0%")
    .add_yaxis("净利润", net_profit, gap="0%")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
                     datazoom_opts=opts.DataZoomOpts(),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} /亿")),)
)

In [213]:
profit_chart.render_notebook()

In [218]:
balance_df=finance.run_query(query(finance.STK_BALANCE_SHEET)\
                  .filter(finance.STK_BALANCE_SHEET.code=='600519.XSHG',\
                         finance.STK_BALANCE_SHEET.report_type==0,\
                         finance.STK_BALANCE_SHEET.report_date.ilike('_____12-31')))\
[['report_date','total_assets','total_current_assets','total_non_current_assets',
  'total_liability','total_current_liability','total_non_current_liability','total_owner_equities']]

In [219]:
balance_df['year'] = balance_df['report_date'].apply(lambda x:str(x.year))
total_assets = list(round(balance_df['total_assets']/100000000,2))
total_current_assets = list(round(balance_df['total_current_assets']/100000000,2))
total_non_current_assets = list(round(balance_df['total_non_current_assets']/100000000,2))
total_liability = list(round(balance_df['total_liability']/100000000,2))
total_current_liability = list(round(balance_df['total_current_liability']/100000000,2))
total_non_current_liability = list(round(balance_df['total_non_current_liability']/100000000,2))
total_owner_equities = list(round(balance_df['total_owner_equities']/100000000,2))

<ipython-input-219-872a2370107d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_df['year'] = balance_df['report_date'].apply(lambda x:str(x.year))


In [236]:
balance_chart = (
    Bar()
    .add_xaxis(years)
    .add_yaxis("总资产", total_assets, gap="0%")
    .add_yaxis("流动资产", total_current_assets, gap="0%")
    .add_yaxis("非流动资产", total_non_current_assets, gap="0%")
    .add_yaxis("总负债", total_liability, gap="0%")
    .add_yaxis("流动负债", total_current_liability, gap="0%")
    .add_yaxis("非流动负债", total_non_current_liability, gap="0%")
    .add_yaxis("股东权益", total_owner_equities, gap="0%")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
                     datazoom_opts=opts.DataZoomOpts(),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} /亿")),)
)

In [221]:
balance_chart.render_notebook()

In [168]:
cashflow_df=finance.run_query(query(finance.STK_CASHFLOW_STATEMENT)\
                  .filter(finance.STK_CASHFLOW_STATEMENT.code=='600519.XSHG',\
                         finance.STK_CASHFLOW_STATEMENT.report_type==0,\
                         finance.STK_CASHFLOW_STATEMENT.report_date.ilike('_____12-31')))\
[['report_date','net_operate_cash_flow','net_invest_cash_flow','net_finance_cash_flow',
  'cash_equivalent_increase','cash_and_equivalents_at_end']]

In [222]:
cashflow_df['year']= cashflow_df['report_date'].apply(lambda x:str(x.year))
net_operate_cash_flow = list(round(cashflow_df['net_operate_cash_flow']/100000000,2))
net_invest_cash_flow = list(round(cashflow_df['net_invest_cash_flow']/100000000,2))
net_finance_cash_flow = list(round(cashflow_df['net_finance_cash_flow']/100000000,2))
cash_equivalent_increase = list(round(cashflow_df['cash_equivalent_increase']/100000000,2))
cash_and_equivalents_at_end = list(round(cashflow_df['cash_and_equivalents_at_end']/100000000,2))

In [223]:
from pyecharts.charts import Bar, Line

line = (
    Line()
    .add_xaxis(xaxis_data=cashflow_df['year'].tolist())
    .add_yaxis(
        series_name="现金及现金等价物净增加额",
        y_axis=cash_equivalent_increase,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="年末现金及现金等价物",
        y_axis=cash_and_equivalents_at_end,
        label_opts=opts.LabelOpts(is_show=False),
    )
)

In [231]:
cashflow_bar = (
    Bar()
    .add_xaxis(cashflow_df['year'].tolist())
    .add_yaxis("经营活动产生的现金流量净额经营", total_assets, gap="0%")
    .add_yaxis("投资活动产生的现金流量净额", total_current_assets, gap="0%")
    .add_yaxis("筹资活动产生的现金流量净额", total_non_current_assets, gap="0%")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(datazoom_opts=opts.DataZoomOpts(),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value}亿")),)
)

In [233]:
cashflow_chart = cashflow_bar.overlap(line)

In [208]:
from pyecharts.charts import Tab

In [237]:
tab = Tab()
tab.add(profit_chart, "利润表")
tab.add(balance_chart, "资产负债表")
tab.add(cashflow_chart, "现金流量表")
tab.render_notebook()

In [244]:
tab.render_embed()

'<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="UTF-8">\n    <title>Awesome-pyecharts</title>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/echarts.min.js"></script>\n\n    \n</head>\n<body>\n        <style>\n        .tab {\n            overflow: hidden;\n            border: 1px solid #ccc;\n            background-color: #f1f1f1;\n        }\n\n        .tab button {\n            background-color: inherit;\n            float: left;\n            border: none;\n            outline: none;\n            cursor: pointer;\n            padding: 12px 16px;\n            transition: 0.3s;\n        }\n\n        .tab button:hover {\n            background-color: #ddd;\n        }\n\n        .tab button.active {\n            background-color: #ccc;\n        }\n\n        .chart-container {\n            display: none;\n            padding: 6px 12px;\n            border-top: none;\n        }\n    </style>\n    <div class="tab">\n            <button class="tablin

In [239]:
profit_df = finance.run_query(query(finance.STK_INCOME_STATEMENT) \
                                      .filter(finance.STK_INCOME_STATEMENT.company_name == '上海汉得信息技术股份有限公司', \
                                              finance.STK_INCOME_STATEMENT.report_type == 0, \
                                              finance.STK_INCOME_STATEMENT.report_date.ilike('_____12-31'))) \
            [['report_date', 'total_operating_revenue', 'total_operating_cost', 'operating_profit', 'total_profit',
              'income_tax', 'net_profit']]

In [246]:
from pyecharts.charts import Page,Grid

In [1]:
from LAC import LAC
from tqdm import tqdm
lac = LAC(mode='lac')

In [ ]:
lac.